# MNIST

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import mnist_data
from random import randint

### 1. 학습 자료 다운로드

In [2]:
mnist = input_data.read_data_sets("./MNIST_data/",one_hot=True)

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
x = tf.placeholder("float",[None,784],name="X-input")
y = tf.placeholder("float",[None,10],name="Y-input")

In [4]:
W = tf.Variable(tf.zeros([784,10]),name="weight")
b = tf.Variable(tf.zeros([10]),name="bias")

In [5]:
W_hist = tf.histogram_summary("weights",W)
b_hist = tf.histogram_summary("biases",b)
y_hist = tf.histogram_summary("y",y)

In [6]:
with tf.name_scope("activation") as scope:
    activation = tf.nn.softmax(tf.matmul(x,W)+b)

learning_rate = 0.9

with tf.name_scope("cost") as scope:
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(activation),reduction_indices=1))
    cost_sum = tf.scalar_summary("cost",cost)
    
with tf.name_scope("train") as scope:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
    
with tf.name_scope("test") as scope:
        # test model
    correct_prediction = tf.equal(tf.argmax(activation, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    accuracy_summary = tf.scalar_summary("accuracy",accuracy)
    

### 학습 시작

In [7]:
training_epoch = 200
display_step = 5
batch_size = 100
init = tf.initialize_all_variables()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [8]:
with tf.Session() as sess:
    # 텐서보드 합치기
    merged = tf.merge_all_summaries()
    writer = tf.train.SummaryWriter("./logs/tuto",sess.graph)
    
    sess.run(init)
    for epoch in range(training_epoch):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples/batch_size)
        
        if epoch % 10 == 0 : #텐서보드에 기록하기
            feed = {x: mnist.test.images, y: mnist.test.labels}
            summary, _ = sess.run([merged,accuracy], feed_dict=feed)
            writer.add_summary(summary,epoch)
        
        else: #학습하기
            for i in range(total_batch):
                batch_xs, batch_ys = mnist.train.next_batch(batch_size)
                sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys})
                
    print("최적화 완료!!")
    

    print ("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))
    
    # test sample
    r = randint(0,mnist.test.num_examples-1)
    print("r:",mnist.test.images[r:r+1])
    print("Label: ",sess.run(tf.argmax(mnist.test.labels[r:r+1],1)))
    print("prediction: ",sess.run(tf.argmax(activation,1),{x:mnist.test.images[r:r+1]}))

InvalidArgumentError: Nan in summary histogram for: HistogramSummary_1
	 [[Node: HistogramSummary_1 = HistogramSummary[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](HistogramSummary_1/tag, bias/read)]]

Caused by op 'HistogramSummary_1', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python3.5/dist-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/usr/local/lib/python3.5/dist-packages/t

### 검증